In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Melon 100 차트 URL
base_url = 'https://www.melon.com/chart/index.htm'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Referer': 'https://www.melon.com/index.htm'
}

# 페이지 요청 및 파싱
response = requests.get(base_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# 곡 정보가 담긴 리스트 가져오기
songs = soup.select('tr[data-song-no]')

# 각 곡의 정보 추출
song_data = []
for song in songs:
    print(song)
    song_id = song['data-song-no']
    title = song.select_one('div.ellipsis.rank01 > span > a').text
    artist = song.select_one('div.ellipsis.rank02 > a').text
    album = song.select_one('div.ellipsis.rank03 > a').text

    # 곡의 상세 정보 페이지 요청 및 파싱
    detail_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
    detail_response = requests.get(detail_url, headers=headers)
    detail_soup = BeautifulSoup(detail_response.text, 'html.parser')

    release_date = detail_soup.select_one('dl.list > dd:nth-of-type(2)').text
    genre = detail_soup.select_one('dl.list > dd:nth-of-type(3)').text
    likes = detail_soup.select_one('span.cnt').text.strip('총건수').replace(',', '').strip()

    lyrics = detail_soup.select_one('div.lyric')
    if lyrics:
        lyrics = lyrics.text.strip()
    else:
        lyrics = ''

    song_data.append({
        'title': title,
        'artist': artist,
        'album': album,
        'release_date': release_date,
        'genre': genre,
        'likes': likes,
        'lyrics': lyrics
    })

# Pandas DataFrame으로 변환
df = pd.DataFrame(song_data)

# 결과를 CSV 파일로 저장
df.to_csv('melon_top_100.csv', index=False, encoding='utf-8-sig')

# 결과 출력 (선택사항)
print(df)